# Exporting American Movie Box Office Hits 

### boxofficemojo.com data acquisition

In [110]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import dateutil.parser
import time
import random


# `movies_urls` list
Use the link structure of offset to add 100 to get the next page. 

In [125]:
offset = 0
base_url = 'https://www.boxofficemojo.com/genre/sg1625223425/?offset='

movies_urls = []

for url in range(10):
    movies_urls.append(base_url + str(offset))
    offset = offset + 100

# `movies_dict`
Get link stub from index page.

In [167]:
def scrape_movies(url, movies_data):
    response = requests.get(url)
    status = response.status_code
    if status == 200:
        print('Status is 200') # double check code runs for nine pages
        page = response.text
        soup = bs(page, "lxml")
        table = soup.find("table")
        rows = [row for row in table.find_all('tr')] 
        for row in rows:
            movies_data.append(row)
    else:
        print(f"Oops! Received status code {status}")
        return None
    
movies_data = []

for movie_url in movies_urls:
    scrape_movies(movie_url, movies_data)
    
# movies_data


Status is 200
Status is 200
Status is 200
Status is 200
Status is 200
Status is 200
Status is 200
Status is 200
Status is 200
Status is 200


In [168]:
# pop first element of each soup which has the header names in the row
movies_data.pop(0)

movies_dict = {}

for row in movies_data:
    items = row.find_all('td')
    try:
        link = items[1].find('a')
        title, url = link.text, link['href']
        movies_dict[title] = [url] + [i.text for i in items]
    except IndexError:
        continue
        

# print(movies_dict)


# `adaptation_movies_df` 

In [151]:
# df using movies dict from above, including link_stub

adaptation_movies_df = pd.DataFrame(movies_dict).T  #transpose
adaptation_movies_df.columns = ['link_stub', 'rank', 'movie_title', 
                    'lifetime_gross', 'max_theaters', 'opening_weekend', 
                    'opening_theathers', 'release_date', 'distributor']
adaptation_movies_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 945 entries, The Avengers to Amityville: The Awakening
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   link_stub          945 non-null    object
 1   rank               945 non-null    object
 2   movie_title        945 non-null    object
 3   lifetime_gross     945 non-null    object
 4   max_theaters       945 non-null    object
 5   opening_weekend    945 non-null    object
 6   opening_theathers  945 non-null    object
 7   release_date       945 non-null    object
 8   distributor        945 non-null    object
dtypes: object(9)
memory usage: 73.8+ KB


In [169]:
adaptation_movies_df.head()

,link_stub,rank,movie_title,lifetime_gross,max_theaters,opening_weekend,opening_theathers,release_date_x,distributor_x,domestic_total_gross,international_total_gross,worldwide_total_gross,domestic_opening,budget,release_date_y,runtime,distributor_y,genres,rating
101 Dalmatians,/title/tt0115433/?ref_=bo_ge_table_49,149,101 Dalmatians,"$136,189,294","2,901","$33,504,025","2,794","Nov 27, 1996",Walt Disney Studios Motion Pictures\n\n,136189294,184500000,320689294,33504025,0,1996-11-27,103.0,Walt Disney Studios Motion Pictures,"Adventure, Comedy, Crime, Family",None
102 Dalmatians,/title/tt0211181/?ref_=bo_ge_table_35,335,102 Dalmatians,"$66,957,026","2,704","$19,883,351","2,704","Nov 22, 2000",Walt Disney Studios Motion Pictures\n\n,66957026,116654745,183611771,19883351,85000000,2000-11-22,100.0,Walt Disney Studios Motion Pictures,"Adventure, Comedy, Family",None
12 Monkeys,/title/tt0114746/?ref_=bo_ge_table_85,385,12 Monkeys,"$57,141,459","1,629","$184,776",3,"Dec 29, 1995",Universal Pictures\n\n,57141459,111698000,168839459,184776,0,1995-12-29,129.0,Universal Pictures,"Mystery, Sci-Fi, Thriller",R
13 Sins,/title/tt2059171/?ref_=bo_ge_table_56,956,13 Sins,"$13,809",45,"$9,261",45,"Apr 18, 2014",RADiUS-TWC\n\n,13809,813104,826913,9261,0,2014-04-11,93.0,RADiUS-TWC,"Horror, Thriller",R
1984,/title/tt0087803/?ref_=bo_ge_table_8,808,1984,"$8,430,492",295,"$29,897",1,"Dec 14, 1984",The Samuel Goldwyn Company\n\n,8430492,8430492,0,29897,0,1984-12-14,113.0,The Samuel Goldwyn Company,"Drama, Sci-Fi",None


# helper functions
# `money_to_int()`  `runtime_to_minutes()`  `to_date()`

In [152]:
# helper functions parse strings into appropriate data types

def money_to_int(moneystring):
    moneystring = moneystring.replace('$', '').replace(',', '')
    return int(moneystring)

def runtime_to_minutes(runtimestring):
    runtime = runtimestring.split()
    try:
        minutes = int(runtime[0])*60 + int(runtime[2])
        return minutes
    except:
        return None

def to_date(datestring):
    date = dateutil.parser.parse(datestring)
    return date


# `get_movie_value()`

In [153]:
# function to scrape values from each movie page, will pass field_name in next function

def get_movie_value(soup, field_name):
 
    obj = soup.find(text=re.compile(field_name))
    
    if not obj: 
        return None
    
    next_element = obj.findNext()
    
    if next_element:
        return next_element.text 
    else:
        return None

# `get_movie_dict()`

In [154]:
# create movie dict to collect values from get_movie_value function from above

def get_movie_dict(link):
    
    base_url = 'https://www.boxofficemojo.com'
    
    # create full url to scrape
    url = base_url + link
    
    # request HTML and parse
    response = requests.get(url)
    status = response.status_code
    if status == 200:
        page = response.text
        soup = bs(page, "lxml")
    else:
        print(f"Oops! Received status code {status}")
        
    headers = ['movie_title', 
               'domestic_total_gross',
               'international_total_gross',
               'worldwide_total_gross',
               'domestic_opening',
               'budget',
               'release_date',
               'runtime',
               'distributor',
               'genres',
               'rating'
              ]

    #get title
    title_string = soup.find('title').text
    title = title_string.split('-')[0].strip()
    print(title) # print movie title to make sure code is running

    '''
    Get GROSS VALUES from performance summary table on left side of each movie page
    '''
    # domestic $$$
    try:     
        raw_domestic_total_gross = (soup.find(class_='mojo-performance-summary-table')
                                        .find_all('span', class_='money')[0].text)
        # convert raw to int       
        domestic_total_gross = money_to_int(raw_domestic_total_gross)
    except:
        domestic_total_gross= 0
    
    # international $$$
    try: 
        raw_international_total_gross = (soup.find(class_='mojo-performance-summary-table')
                                        .find_all('span', class_='money')[1].text)
        #convert raw to int    
        international_total_gross = money_to_int(raw_international_total_gross)
    except:
         international_total_gross = 0

    # worldwide $$$
    try: 
        raw_worldwide_total_gross = (soup.find(class_='mojo-performance-summary-table')
                                        .find_all('span', class_='money')[2]
                                        .text)
        # convert raw to int
        worldwide_total_gross = money_to_int(raw_worldwide_total_gross)

    except:
        worldwide_total_gross = 0

    '''
    Get MOVIE DETAILS from summary details table in center of each movie page
    '''    
    # domestic opening $$$ 
    try: 
        raw_domestic_opening = get_movie_value(soup,'Domestic Opening')
        
        # convert to int
        domestic_opening = money_to_int(raw_domestic_opening)
    except:
        domestic_opening = 0
    
    # budget $$$
    try: 
        raw_budget = get_movie_value(soup,'Budget')
        
        # convert budget to int
        budget = money_to_int(raw_budget)
    except:
        budget = 0
    
    # release date
    try: 
        raw_release_date = get_movie_value(soup,'Release Date').split('\n')[0]
        # convert date to dt
        release_date = to_date(raw_release_date)
    except:
        release_date = 0

    # runtime 
    try: 
        raw_runtime = get_movie_value(soup,'Running')
        # convert runtime to minutes
        runtime = runtime_to_minutes(raw_runtime)
    except:
        runtime = 0 
    
    # distributor
    try:  
        raw_distributor = get_movie_value(soup,'Distributor')
        # clean up 
        distributor = raw_distributor.replace('See full company information', '').replace('\n\n', '')
    except:
        distributor = 'NA' 

    # genres
    try: 
        genres = get_movie_value(soup,'Genres').replace('\n    \n        ', ', ')
    except:
        genres = 'NA'

    # rating    
    try: 
        rating = get_movie_value(soup,'MPAA')
    except:
        rating = 'NA'
    
    # movie dictionary
    movie_dict = dict(zip(headers, [title,
                                    domestic_total_gross,
                                    international_total_gross,
                                    worldwide_total_gross,
                                    domestic_opening,
                                    budget,
                                    release_date,
                                    runtime,
                                    distributor,
                                    genres,
                                    rating]))
    
#     print(movie_dict)

    return movie_dict

# `adaptation_movie_page_info_list`

In [155]:
# loop to pass link using get_movie_dict() function from above

adaptation_movie_page_info_list = []

for link in adaptation_movies_df.link_stub:
    adaptation_movie_page_info_list.append(get_movie_dict(link))

The Avengers
The Lion King
The Dark Knight
Beauty and the Beast
Avengers: Age of Ultron
The Dark Knight Rises
Captain Marvel
The Hunger Games: Catching Fire
Wonder Woman
Iron Man 3
Captain America: Civil War
The Hunger Games
Spider
Transformers: Revenge of the Fallen
Spider
Guardians of the Galaxy Vol. 2
Harry Potter and the Deathly Hallows: Part 2
Spider
The Jungle Book
Deadpool
Jurassic Park
Transformers: Dark of the Moon
The Hunger Games: Mockingjay
Spider
Joker
Aquaman
Spider
Alice in Wonderland
Guardians of the Galaxy
Batman v Superman: Dawn of Justice
Suicide Squad
Transformers
Iron Man
Deadpool 2
Harry Potter and the Sorcerer's Stone
Thor: Ragnarok
Iron Man 2
Harry Potter and the Half
The Twilight Saga: Eclipse
The Twilight Saga: New Moon
Harry Potter and the Deathly Hallows: Part 1
The Twilight Saga: Breaking Dawn
Harry Potter and the Order of the Phoenix
The Chronicles of Narnia: The Lion, the Witch and the Wardrobe
Man of Steel
Harry Potter and the Goblet of Fire
The Hunger G

Timecop
Red Dawn
The Three Stooges
The Sisterhood of the Traveling Pants 2
The Beverly Hillbillies
Speed Racer
Prom Night
Always
Underdog
Pokémon the Movie 2000
Need for Speed
Star Trek: Nemesis
Biloxi Blues
Death at a Funeral
The Lizzie McGuire Movie
Widows
Resident Evil: Retribution
Teenage Mutant Ninja Turtles III
Fever Pitch
Dirty Rotten Scoundrels
Thir13en Ghosts
Homeward Bound: The Incredible Journey
The Hills Have Eyes
The Angry Birds Movie 2
Valerian and the City of a Thousand Planets
The Mirror Has Two Faces
The Water Horse
Pee
Paddington 2
House on Haunted Hill
Love, Simon
Max Payne
Three Fugitives
Ghost in the Shell
Peter Rabbit 2: The Runaway
The Fly
2010: The Year We Make Contact
Catwoman
Resident Evil
The Wild Thornberrys
The Ladykillers
Hitman
30 Days of Night
The Texas Chainsaw Massacre: The Beginning
Rugrats Go Wild
The Grudge 2
The Crazies
The Sisterhood of the Traveling Pants
Primary Colors
The Ghost and the Darkness
Surrogates
Last Holiday
Mighty Morphin Power Range

SubUrbia
The House of Yes
Nancy Drew and the Hidden Staircase
The Celestine Prophecy
Noah's Arc: Jumping the Broom
Tape
The Browning Version
DOA: Dead or Alive
Body Snatchers
Night of the Living Dead 3D
Kingsglaive: Final Fantasy XV
The Return of Swamp Thing
We Are the Best!
The Cherry Orchard
Oleanna
Wo zhi nv ren xin
I Spit on Your Grave
We Are What We Are
Brenda Starr
Gumby: The Movie
The War at Home
Madame Bovary
Blood Ties
Happy New Year
Rurouni Kenshin Part I: Origins
The Substance of Fire
Maniac
Tiger Eyes
The Sweeney
Silent Night
The 24th Day
13 Sins
War of the Buttons
Radio Free Albemuth
Human Capital
Come Out and Play
Postman Pat: The Movie
Billy and Buddy
I Spit on Your Grave 2
Amityville: The Awakening


In [170]:
# adaptation_movie_page_info_list

# `adaptation_movie_info_df` from `adaptation_movie_page_info_list`

In [157]:
# #convert list of dict to df

adaptation_movie_info_df = pd.DataFrame(adaptation_movie_page_info_list)  
adaptation_movie_info_df.set_index('movie_title', inplace=True)

adaptation_movie_info_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 945 entries, The Avengers to Amityville: The Awakening
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   domestic_total_gross       945 non-null    int64         
 1   international_total_gross  945 non-null    int64         
 2   worldwide_total_gross      945 non-null    int64         
 3   domestic_opening           945 non-null    int64         
 4   budget                     945 non-null    int64         
 5   release_date               945 non-null    datetime64[ns]
 6   runtime                    931 non-null    float64       
 7   distributor                945 non-null    object        
 8   genres                     945 non-null    object        
 9   rating                     774 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(5), object(3)
memory usage: 81.2+ KB


# merge dfs: <br/>
# `adaptation_movies_df` + `adaptation_movie_info_df` 

In [159]:
# match this back up with the movie information collected from the table by merging  dataframes
adaptation_movies_df = adaptation_movies_df.merge(adaptation_movie_info_df, left_index=True, right_index=True)
adaptation_movies_df

,link_stub,rank,movie_title,lifetime_gross,max_theaters,opening_weekend,opening_theathers,release_date_x,distributor_x,domestic_total_gross,international_total_gross,worldwide_total_gross,domestic_opening,budget,release_date_y,runtime,distributor_y,genres,rating
101 Dalmatians,/title/tt0115433/?ref_=bo_ge_table_49,149,101 Dalmatians,"$136,189,294","2,901","$33,504,025","2,794","Nov 27, 1996",Walt Disney Studios Motion Pictures\n\n,136189294,184500000,320689294,33504025,0,1996-11-27,103.0,Walt Disney Studios Motion Pictures,"Adventure, Comedy, Crime, Family",None
102 Dalmatians,/title/tt0211181/?ref_=bo_ge_table_35,335,102 Dalmatians,"$66,957,026","2,704","$19,883,351","2,704","Nov 22, 2000",Walt Disney Studios Motion Pictures\n\n,66957026,116654745,183611771,19883351,85000000,2000-11-22,100.0,Walt Disney Studios Motion Pictures,"Adventure, Comedy, Family",None
12 Monkeys,/title/tt0114746/?ref_=bo_ge_table_85,385,12 Monkeys,"$57,141,459","1,629","$184,776",3,"Dec 29, 1995",Universal Pictures\n\n,57141459,111698000,168839459,184776,0,1995-12-29,129.0,Universal Pictures,"Mystery, Sci-Fi, Thriller",R
13 Sins,/title/tt2059171/?ref_=bo_ge_table_56,956,13 Sins,"$13,809",45,"$9,261",45,"Apr 18, 2014",RADiUS-TWC\n\n,13809,813104,826913,9261,0,2014-04-11,93.0,RADiUS-TWC,"Horror, Thriller",R
1984,/title/tt0087803/?ref_=bo_ge_table_8,808,1984,"$8,430,492",295,"$29,897",1,"Dec 14, 1984",The Samuel Goldwyn Company\n\n,8430492,8430492,0,29897,0,1984-12-14,113.0,The Samuel Goldwyn Company,"Drama, Sci-Fi",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yogi Bear,/title/tt1302067/?ref_=bo_ge_table_32,232,Yogi Bear,"$100,246,011","3,515","$16,411,322","3,515","Dec 17, 2010",Warner Bros.\n\n,100246011,103263363,203509374,16411322,80000000,2010-12-17,81.0,Warner Bros.,"Adventure, Animation, Comedy, Family",PG
You've Got Mail,/title/tt0128853/?ref_=bo_ge_table_86,186,You've Got Mail,"$115,821,495","2,948","$18,426,749","2,691","Dec 18, 1998",Warner Bros.\n\n,115821495,135000000,250821495,18426749,65000000,1998-12-18,119.0,Warner Bros.,"Comedy, Drama, Romance",PG
"Yours, Mine & Ours",/title/tt0443295/?ref_=bo_ge_table_5,405,"Yours, Mine & Ours","$53,412,862","3,210","$17,461,108","3,206","Nov 23, 2005",Paramount Pictures\n\n,53412862,19250061,72662923,17461108,45000000,2005-11-23,88.0,Paramount Pictures,"Comedy, Family, Romance",PG
Zathura: A Space Adventure,/title/tt0406375/?ref_=bo_ge_table_77,577,Zathura: A Space Adventure,"$29,258,869","3,232","$13,427,872","3,223","Nov 11, 2005",Sony Pictures Entertainment (SPE)\n\n,29258869,35820235,65079104,13427872,65000000,2005-11-11,101.0,Sony Pictures Entertainment (SPE),"Action, Adventure, Comedy, Family, Fantasy, Sc...",PG


In [160]:
# movie df to start EDA
adaptation_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 901 entries, 101 Dalmatians to Æon Flux
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   link_stub                  901 non-null    object        
 1   rank                       901 non-null    object        
 2   movie_title                901 non-null    object        
 3   lifetime_gross             901 non-null    object        
 4   max_theaters               901 non-null    object        
 5   opening_weekend            901 non-null    object        
 6   opening_theathers          901 non-null    object        
 7   release_date_x             901 non-null    object        
 8   distributor_x              901 non-null    object        
 9   domestic_total_gross       901 non-null    int64         
 10  international_total_gross  901 non-null    int64         
 11  worldwide_total_gross      901 non-null    int64         


In [165]:
# picke df 
adaptation_movies_all_df = adaptation_movies_df
adaptation_movies_all_df.to_pickle('adaptation_movies_all_df.pkl')


In [164]:
# save to csv
adaptation_movies_df.to_csv(r'/Users/sandraparedes/Dropbox/Mac/Downloads/adaptation_movies_all_df.csv', index=False)
